<a href="https://colab.research.google.com/github/shivsharanrupesh/GenAI_Agents_Journey/blob/main/Agents_Tutorials_Basic_Conversational_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Building a Conversational Agent with Context Awareness
Overview**  

This tutorial describes how to build a conversational agent that can keep track of conversations from one interaction to the next. We use a modern AI framework to build an agent capable of more natural and coherent conversations.

 **Motivation**

Lots of simple chatbots do not maintain any context, which often creates angering experiences for the user. Our goal in this tutorial is to remedy this consideration by constructing a conversational agent that can recall parts of the conversation and refer to them later to enhance the interaction experience.

 **Key Components**

 1. **Language Model**: The central AI component that generates responses.

 2. **Prompt Template**: Unique structure of how we interact.

 3. **History Manager**: Responsible for managing conversation context as well as historical contexts.

 4. **Message Store**: Responsible for storing messages for each conversation session.


 **Methods and Procedures**

 **Setting Up the Environment**

AI frameworks will be laid out to help start and ensure access to an appropriate language model. This is very much the foundation for our conversation agent.

 **Creating the Chat History Storage**

This sets up a way to manage multiple conversations. Each iteration may be uniquely identified and allowed its own message history.

 **Creating the Format of the Conversation**

 **Create a structure that will utilize:**

1. A system message that specifies the role of the AI
2. A placeholder for historical conversation;
The input provided by the user.

This structure will guide the AI into responding while also keeping a sense of continuity through the conversation.

**Creating the Conversational Chain**

Combine the converse template with the language model to create a simple conversational chain. Now wrap this chain with a history manager that automatically takes care of insertion and retrieval of conversation history.

**Interacting with the Agent**

The agent can be used by invoking it with a user input and a session identifier. The history manager is responsible for fetching the conversation history to be inserted into the prompt and storing incoming messages after every interaction.

 **Conclusion**

This methodology of developing a conversational agent has several advantages:
1. **Context awareness** allows the agent to refer to previous portions of the conversation, promoting more natural interactions.
2. **Simplicity**: Modular design allows for basic implementation.
3. **Flexibility:** Easy to change conversational structures or switch the underlying language model.
4. **Scalability:** A session-based approach allows for dealing with several independent conversations.

 **Further improvement can be implemented in the agent, guided by:**

1. Further prompt engineering.
2. Integration with external knowledge bases.
3. Adding specialized capabilities for particular domains.
4. Incorporating error handling and conversation repair strategies.

This design fundamentally builds upon the premise of context management enabling this conversational agent to vastly improve upon very basic chatbot functionality and thus, open up avenues for more engaging and helpful AI assistants.


 **Conversational Agent Tutorial**

In this notebook, we shall demonstrate the building of a simple conversational agent using LangChain.

 **Import Libraries:**

In [3]:
#pip install langchain langchain_experimental openai python-dotenv langchain_openai

  Using cached langchain_experimental-0.3.4-py3-none-any.whl.metadata (1.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.35
    Uninstalling langchain-core-0.3.35:
      Successfully uninstalled langchain-core-0.3.35
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.18
    Uninstalling langchain-0.3.18:
      Successfully uninstalled langchain-0.3.18


In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from openai import OpenAI
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

 **Initialize the Model**

In [15]:
client = OpenAI(api_key=api_key)
llm = ChatOpenAI(
    model="gpt-4",
    api_key=api_key,
    max_tokens=500,
    temperature=0
)

 **Create a Simple in-memory store for chat histories**

In [24]:
store = {}

def get_chat_history(session_id):
    """
    Retrieves or creates a chat history for a given session ID.

    This function manages chat histories for multiple sessions using an in-memory store.
    If the session ID does not already exist in the store, a new `ChatMessageHistory`
    object is created and associated with the session ID. Otherwise, the existing
    chat history for the session is returned.

    Args:
        session_id (str): A unique identifier for the session. This is used to
                          retrieve or create the corresponding chat history.

    Returns:
        ChatMessageHistory: A `ChatMessageHistory` object containing the chat history
                           for the specified session. If the session ID is new, an
                           empty `ChatMessageHistory` object is returned.

    Example:
        >>> session_id = "user_123"
        >>> chat_history = get_chat_history(session_id)
        >>> chat_history.add_user_message("Hello!")
        >>> chat_history.add_ai_message("Hi there!")
        >>> print(chat_history.messages)
        [HumanMessage(content="Hello!"), AIMessage(content="Hi there!")]
    """
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

**Create the prompt template**

In [25]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

**Combine the prompt and model into a runnable chain**

In [26]:
chain = prompt | llm

**Wrap the chain with message history**

In [33]:
"""
A chain enhanced with chat history functionality.

This wrapper adds the ability to maintain and utilize chat history for a given session.
It retrieves or creates a chat history for each session using the `get_chat_history`
function and ensures that the history is passed to the chain for context-aware responses.

Args:
    chain: The base chain (e.g., a chatbot or LLM pipeline) to enhance with chat history.
    get_chat_history: A function that retrieves or creates a chat history for a session.
    input_messages_key (str): The key in the input dictionary where the user's message is located.
    output_messages_key (str): The key in the output dictionary where the chat history is stored.

Example:
    >>> response = chain_with_history.invoke(
    ...     {"input": "Hello!"},
    ...     config={"configurable": {"session_id": "user_123"}}
    ... )
    >>> print(response)
    {"output": "Hi there!", "history": [...]}
"""

chain_with_history = RunnableWithMessageHistory(
    chain, get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [37]:
response_1 = chain_with_history.invoke(
    {"input": "Hello! How's it going, whats whether out there? "},
    config={"configurable": {"session_id": "user_123"}}
)
print("AI:", response_1.content)

response_2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": "user_123"}}
)
print("AI:", response_2.content)

AI: I'm an artificial intelligence and don't have the ability to perceive the environment or weather. However, you can check the weather in your area by using a weather app or website. Can I assist you with anything else?
AI: Your previous message was: "Hello! How's it going, whats whether out there?"


 **Print the conversation history**

In [40]:
print("\n Convseration History: ")
for message in store["user_123"].messages:
    print(f"{message.type}: {message.content}")


 Convseration History: 
human: Hello!
ai: Hello! How can I assist you today?
human: What was my old message!
ai: I'm sorry, but as an AI, I don't have the ability to access or retrieve past interactions unless they are part of the current conversation. How can I assist you further?
human: Hello! How's it going, whats whether out there? 
ai: As an artificial intelligence, I don't have the ability to experience weather or check real-time conditions. However, you can check the weather by using a weather app or website, or by asking a voice-activated device like Google Home or Amazon Echo, if you have one.
human: What was my old message!
ai: I'm sorry for any confusion, but as an AI, I don't have the ability to access or retrieve past interactions unless they are part of the current conversation. Can I assist you with anything else?
human: Hello! How's it going, whats whether out there? 
ai: I'm an artificial intelligence and don't have the ability to perceive the environment or weather. 